# WoW Token - Prediction Models

O objetivo desse notebook é realizar a criação, configuração e backtesting de estratégias de previsão e aplicá-las à uma estratégia de investimento, para ver o retorno potencial tanto em gold quanto em dólares. Foi utilizado também estratégias de insights de dados (encontrados no notebook de Data Science) como: necessidade de normalização da entrada para facilitar o cálculo da previsão e transformar a data em datetime do Python, para facilitar a filtragem em uma época específica.

In [1]:
matplotlib inline

In [2]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
wowTokenAppendedData = []
regions = ['NA', 'EU', 'CN', 'KR', 'TW']

for entry in os.scandir('./input/wowtoken'):
    if entry.is_file():
        wowTokenEntry = pd.read_csv(entry.path)
        wowTokenEntry['region'] = (os.path.splitext(entry.name)[0])
        wowTokenEntry['date'] = pd.to_datetime(wowTokenEntry['date'])
        wowTokenAppendedData.append(wowTokenEntry)
        
data = pd.concat(wowTokenAppendedData)

In [4]:
data.pivot(columns='region', values='price').describe()

region,CN,EU,KR,NA,TW
count,6835.000000,7744.000000,7642.000000,7886.000000,7468.000000
mean,242416.955377,164289.345687,271204.557969,95573.815876,260885.958356
std,168907.809419,101425.233611,98936.313849,64075.637750,104487.473796
min,48604.000000,30352.000000,121305.000000,18296.000000,114619.000000
25%,83650.000000,68769.250000,179645.500000,37004.250000,174211.000000
50%,215554.000000,171327.000000,285099.500000,89460.500000,213519.000000
75%,400670.000000,256468.750000,350899.250000,163553.250000,335554.500000
max,586090.000000,401827.000000,595930.000000,238572.000000,501220.000000


In [5]:
data2018 = data.loc[data['date'].dt.year >= 2018].reset_index()
data2018.pivot(columns='region', values='price').describe()

region,CN,EU,KR,NA,TW
count,1983.000000,2076.000000,2088.000000,2085.000000,2085.000000
mean,414124.769037,272073.894990,365853.373084,165006.325659,381305.507434
std,129721.911793,68360.274879,76882.410805,41141.510822,75199.125368
min,206304.000000,168106.000000,269461.000000,100993.000000,250412.000000
25%,229674.000000,184120.000000,298436.000000,111341.000000,284082.000000
50%,496259.000000,298415.500000,322257.500000,182392.000000,413693.000000
75%,507332.500000,336512.500000,441929.750000,199718.000000,441348.000000
max,586090.000000,401827.000000,595930.000000,238572.000000,501220.000000


In [6]:
data['unix_timestamp'] = data['date'].astype(np.int64) // 10**9
data.head()

,date,price,region,unix_timestamp
0,2015-05-06 22:04:08,250000,TW,1430949848
1,2015-05-07 02:04:09,275180,TW,1430964249
2,2015-05-07 06:04:01,308756,TW,1430978641
3,2015-05-07 10:04:03,342634,TW,1430993043
4,2015-05-07 14:05:10,328547,TW,1431007510


In [7]:
dataNA = data.loc[data['region'] == 'NA'].drop(['date', 'region'], axis=1)
dataNA.head()

,price,unix_timestamp
0,30000,1428426758
1,30906,1428444242
2,28515,1428458642
3,25024,1428473042
4,24038,1428487477


In [8]:
dataCN = data.loc[data['region'] == 'CN'].drop(['date', 'region'], axis=1)
dataCN.head()

,price,unix_timestamp
0,48604,1430237030
1,61104,1430251200
2,66164,1430265599
3,73054,1430279997
4,70756,1430294397


In [9]:
dataEU = data.loc[data['region'] == 'EU'].drop(['date', 'region'], axis=1)
dataEU.head()

,price,unix_timestamp
0,35000,1429628705
1,41041,1429643104
2,44392,1429657504
3,41265,1429671904
4,40361,1429686304


In [10]:
dataKR = data.loc[data['region'] == 'KR'].drop(['date', 'region'], axis=1)
dataKR.head()

,price,unix_timestamp
0,123523,1430956079
1,138673,1430970314
2,155646,1430984701
3,174452,1430999118
4,179163,1431013518


In [11]:
dataTW = data.loc[data['region'] == 'TW'].drop(['date', 'region'], axis=1)
dataTW.head()

,price,unix_timestamp
0,250000,1430949848
1,275180,1430964249
2,308756,1430978641
3,342634,1430993043
4,328547,1431007510


In [12]:
def normalize_price(prices):
    minPrice = min(prices)
    maxPrice = max(prices)
    
    normalized_prices = list(map(lambda p: (p-minPrice)/(maxPrice-minPrice), prices))
    
    return normalized_prices

In [13]:
dataNA['norm_price'] = normalize_price(dataNA['price'])
dataCN['norm_price'] = normalize_price(dataCN['price'])
dataEU['norm_price'] = normalize_price(dataEU['price'])
dataKR['norm_price'] = normalize_price(dataKR['price'])
dataTW['norm_price'] = normalize_price(dataTW['price'])

In [14]:
dataNA['norm_price'].describe()

count    7886.000000
mean        0.350823
std         0.290888
min         0.000000
25%         0.084931
50%         0.323070
75%         0.659433
max         1.000000
Name: norm_price, dtype: float64

In [15]:
dataCN['norm_price'].describe()

count    6835.000000
mean        0.360592
std         0.314255
min         0.000000
25%         0.065204
50%         0.310613
75%         0.655024
max         1.000000
Name: norm_price, dtype: float64

In [16]:
dataEU['norm_price'].describe()

count    7744.000000
mean        0.360555
std         0.273034
min         0.000000
25%         0.103418
50%         0.379501
75%         0.608700
max         1.000000
Name: norm_price, dtype: float64

In [17]:
dataKR['norm_price'].describe()

count    7642.000000
mean        0.315827
std         0.208452
min         0.000000
25%         0.122919
50%         0.345103
75%         0.483738
max         1.000000
Name: norm_price, dtype: float64

In [18]:
dataTW['norm_price'].describe()

count    7468.000000
mean        0.378341
std         0.270272
min         0.000000
25%         0.154143
50%         0.255819
75%         0.571482
max         1.000000
Name: norm_price, dtype: float64

## Basic Model

This model will predict the next price rise/decrease using the last price variation.

In [19]:
class BasicModel:
    def __init__(self):
        self.currentPrediction = 0 # 1 for rise, 0 for decrease
        self.lastPrice = 0
        
    def make_next_prediction(self, currentPriceSeries):
        currentPrice = currentPriceSeries[-1]
        self.currentPrediction = 1 if self.lastPrice < currentPrice else 0
        self.lastPrice = currentPrice
        return self.currentPrediction
    
    def measure_accuracy(self, prices):
        realVariation = 0 # 1 for rise, 0 for decrease
        lastPrice = 0
        correctPredictions = 0
        currentPricesSeen = []
        for price in prices:
            realVariation = 1 if lastPrice <= price else 0
            
            currentPricesSeen.append(price)
            pred = self.make_next_prediction(currentPricesSeen)
            
            correctPredictions += 1 if pred == realVariation else 0
            
        return correctPredictions/len(prices)
    
            
basicModel = BasicModel()

In [20]:
basicModelAccuracyNA = basicModel.measure_accuracy(dataNA['norm_price'])
basicModelAccuracyCN = basicModel.measure_accuracy(dataCN['norm_price'])
basicModelAccuracyEU = basicModel.measure_accuracy(dataEU['norm_price'])
basicModelAccuracyKR = basicModel.measure_accuracy(dataKR['norm_price'])
basicModelAccuracyTW = basicModel.measure_accuracy(dataTW['norm_price'])

## Random Model

This model will predict the next price rise/decrease randomly.

In [21]:
np.random.seed(1212)

class RandomModel:
    def __init__(self):
        self.currentPrediction = 0 # 1 for rise, 0 for decrease
        
    def make_next_prediction(self, currentPriceSeries):
        self.currentPrediction = np.random.randint(2) # randomly gets 0 or 1
        return self.currentPrediction
    
    def measure_accuracy(self, prices):
        realVariation = 0 # 1 for rise, 0 for decrease
        lastPrice = 0
        correctPredictions = 0
        currentPricesSeen = []
        for price in prices:
            realVariation = 1 if lastPrice <= price else 0
            
            currentPricesSeen.append(price)
            pred = self.make_next_prediction(currentPricesSeen)
            
            correctPredictions += 1 if pred == realVariation else 0
            
        return correctPredictions/len(prices)
    
            
randomModel = RandomModel()

In [22]:
randomModelAccuracyNA = randomModel.measure_accuracy(dataNA['norm_price'])
randomModelAccuracyCN = randomModel.measure_accuracy(dataCN['norm_price'])
randomModelAccuracyEU = randomModel.measure_accuracy(dataEU['norm_price'])
randomModelAccuracyKR = randomModel.measure_accuracy(dataKR['norm_price'])
randomModelAccuracyTW = randomModel.measure_accuracy(dataTW['norm_price'])

## Simple Moving Average Model

This model will predict the next price rise/decrease based on a SMA (Simple Moving Average).
A SMA is an average of the price in the last N periods.
In this project, I'll use N=10 for short SMA and N=50 for long SMA.

In [23]:
class SMAModel:
    def __init__(self, shortPeriod, longPeriod):
        self.shortPeriod = shortPeriod
        self.longPeriod = longPeriod
        self.shortSMA = 0
        self.longSMA = 0
        self.lastShortSMA = 0
        self.lastLongSMA = 0
        self.currentPrediction = 0 # 1 for rise, 0 for decrease
        
    def calculateSMA(self, data):
        if len(data) < self.longPeriod:
            return
            
        else:
            self.lastShortSMA = self.shortSMA
            self.lastLongSMA = self.longSMA
            self.shortSMA = np.mean(data[-10:])
            self.longSMA = np.mean(data[-50:])
            
            self.analyze_prediction()
            
    def analyze_prediction(self):
        if self.shortSMA > self.lastShortSMA and self.longSMA > self.lastLongSMA:
            # Both SMAs are increasing, so the tendency is to rise
            self.currentPrediction = 1
            
        elif self.shortSMA <= self.lastShortSMA and self.longSMA <= self.lastLongSMA:
            # Both SMAs are decreasing, so the tendency is to fall
            self.currentPrediction = 0
            
        elif self.lastShortSMA <= self.longSMA and self.shortSMA > self.longSMA:
            # The short SMA crossed the long SMA by increasing itself, so in this case,
            # we hope that the short SMA goes back to the long SMA, so the tendency is to fall
            self.currentPrediction = 0
            
        elif self.lastShortSMA > self.longSMA and self.shortSMA <= self.longSMA:
            # The short SMA crossed the long SMA by decreasing itself, so in this case,
            # we hope that the short SMA goes back to the long SMA, so the tendency is to rise
            self.currentPrediction = 1
            
    def make_next_prediction(self, currentPriceSeries):
        if len(currentPriceSeries) <= self.longPeriod:
            return 0
        
        else:
            self.calculateSMA(currentPriceSeries)
            
            return self.currentPrediction
            
            
    def measure_accuracy(self, prices):
        realVariation = 0 # 1 for rise, 0 for decrease
        lastPrice = 0
        correctPredictions = 0
        currentPricesSeen = []
        for price in prices:
            realVariation = 1 if lastPrice <= price else 0
            
            currentPricesSeen.append(price)
            pred = self.make_next_prediction(currentPricesSeen)
            
            correctPredictions += 1 if pred == realVariation else 0
            
        return correctPredictions/len(prices)
            
smaModel = SMAModel(10, 50)

In [24]:
smaModelAccuracyNA = smaModel.measure_accuracy(dataNA['norm_price'])
smaModelAccuracyCN = smaModel.measure_accuracy(dataCN['norm_price'])
smaModelAccuracyEU = smaModel.measure_accuracy(dataEU['norm_price'])
smaModelAccuracyKR = smaModel.measure_accuracy(dataKR['norm_price'])
smaModelAccuracyTW = smaModel.measure_accuracy(dataTW['norm_price'])

## Exponencial Moving Average Model

This model will predict the next price rise/decrease based on a EMA (Exponencial Moving Average).
An EMA is an average of the price in the last N periods with a multiplier that increases the augments current price influence on prediction.
In this project, I'll use N=10 for short EMA and N=50 for long EMA.

In [25]:
class EMAModel:
    def __init__(self, shortPeriod, longPeriod):
        self.shortPeriod = shortPeriod
        self.longPeriod = longPeriod
        self.shortEMA = 0
        self.longEMA = 0
        self.lastShortEMA = 0
        self.lastLongEMA = 0
        self.currentPrediction = 0 # 1 for rise, 0 for decrease
        
    def calculateEMA(self, data):
        if len(data) < self.longPeriod:
            return
            
        else:
            self.lastShortEMA = self.shortEMA
            self.lastLongEMA = self.longEMA
            self.shortEMA = pd.DataFrame(data[-10:]).ewm(span=10, adjust=False).mean().iloc[-1, 0]
            self.longEMA = pd.DataFrame(data[-50:]).ewm(span=50, adjust=False).mean().iloc[-1, 0]
            
            self.analyze_prediction()
            
    def analyze_prediction(self):
        if self.shortEMA > self.lastShortEMA and self.longEMA > self.lastLongEMA:
            # Both EMAs are increasing, so the tendency is to rise
            self.currentPrediction = 1
            
        elif self.shortEMA <= self.lastShortEMA and self.longEMA <= self.lastLongEMA:
            # Both EMAs are decreasing, so the tendency is to fall
            self.currentPrediction = 0
            
        elif self.lastShortEMA <= self.longEMA and self.shortEMA > self.longEMA:
            # The short EMA crossed the long EMA by increasing itself, so in this case,
            # we hope that the short EMA goes back to the long EMA, so the tendency is to fall
            self.currentPrediction = 0
            
        elif self.lastShortEMA > self.longEMA and self.shortEMA <= self.longEMA:
            # The short EMA crossed the long EMA by decreasing itself, so in this case,
            # we hope that the short EMA goes back to the long EMA, so the tendency is to rise
            self.currentPrediction = 1
            
    def make_next_prediction(self, currentPriceSeries):
        if len(currentPriceSeries) <= self.longPeriod:
            return 0
        
        else:
            self.calculateEMA(currentPriceSeries)
            
            return self.currentPrediction
            
            
    def measure_accuracy(self, prices):
        realVariation = 0 # 1 for rise, 0 for decrease
        lastPrice = 0
        correctPredictions = 0
        currentPricesSeen = []
        for price in prices:
            realVariation = 1 if lastPrice <= price else 0
            
            currentPricesSeen.append(price)
            pred = self.make_next_prediction(currentPricesSeen)
            
            correctPredictions += 1 if pred == realVariation else 0
            
        return correctPredictions/len(prices)
            
emaModel = EMAModel(10, 50)

In [26]:
emaModelAccuracyNA = emaModel.measure_accuracy(dataNA['norm_price'])
emaModelAccuracyCN = emaModel.measure_accuracy(dataCN['norm_price'])
emaModelAccuracyEU = emaModel.measure_accuracy(dataEU['norm_price'])
emaModelAccuracyKR = emaModel.measure_accuracy(dataKR['norm_price'])
emaModelAccuracyTW = emaModel.measure_accuracy(dataTW['norm_price'])

## Models Accuracy

In [27]:
print('-'*56)
print('Model\t\t\t\tRegion\t\tAccuracy')
print('-'*56)
print('Basic Prediction Model\t\tNA\t\t%.6f' %basicModelAccuracyNA)
print('Random Prediction Model\t\tNA\t\t%.6f' %randomModelAccuracyNA)
print('SMA Prediction Model\t\tNA\t\t%.6f' %smaModelAccuracyNA)
print('EMA Prediction Model\t\tNA\t\t%.6f' %emaModelAccuracyNA)
print('-'*56)
print('Basic Prediction Model\t\tCN\t\t%.6f' %basicModelAccuracyCN)
print('Random Prediction Model\t\tCN\t\t%.6f' %randomModelAccuracyCN)
print('SMA Prediction Model\t\tCN\t\t%.6f' %smaModelAccuracyCN)
print('EMA Prediction Model\t\tCN\t\t%.6f' %emaModelAccuracyCN)
print('-'*56)
print('Basic Prediction Model\t\tEU\t\t%.6f' %basicModelAccuracyEU)
print('Random Prediction Model\t\tEU\t\t%.6f' %randomModelAccuracyEU)
print('SMA Prediction Model\t\tEU\t\t%.6f' %smaModelAccuracyEU)
print('EMA Prediction Model\t\tEU\t\t%.6f' %emaModelAccuracyEU)
print('-'*56)
print('Basic Prediction Model\t\tKR\t\t%.6f' %basicModelAccuracyKR)
print('Random Prediction Model\t\tKR\t\t%.6f' %randomModelAccuracyKR)
print('SMA Prediction Model\t\tKR\t\t%.6f' %smaModelAccuracyKR)
print('EMA Prediction Model\t\tKR\t\t%.6f' %emaModelAccuracyKR)
print('-'*56)
print('Basic Prediction Model\t\tTW\t\t%.6f' %basicModelAccuracyTW)
print('Random Prediction Model\t\tTW\t\t%.6f' %randomModelAccuracyTW)
print('SMA Prediction Model\t\tTW\t\t%.6f' %smaModelAccuracyTW)
print('EMA Prediction Model\t\tTW\t\t%.6f' %emaModelAccuracyTW)
print('-'*56)

--------------------------------------------------------
Model				Region		Accuracy
--------------------------------------------------------
Basic Prediction Model		NA		0.512300
Random Prediction Model		NA		0.489855
SMA Prediction Model		NA		0.571012
EMA Prediction Model		NA		0.529419
--------------------------------------------------------
Basic Prediction Model		CN		0.485296
Random Prediction Model		CN		0.501683
SMA Prediction Model		CN		0.547037
EMA Prediction Model		CN		0.495538
--------------------------------------------------------
Basic Prediction Model		EU		0.492123
Random Prediction Model		EU		0.498838
SMA Prediction Model		EU		0.585744
EMA Prediction Model		EU		0.525956
--------------------------------------------------------
Basic Prediction Model		KR		0.495289
Random Prediction Model		KR		0.501963
SMA Prediction Model		KR		0.512824
EMA Prediction Model		KR		0.499346
--------------------------------------------------------
Basic Prediction Model		TW		0.492501
Random Predicti

# Investments Results

In [28]:
class Investment:
    def __init__(self, model, initialGold):
        self.initialGold = initialGold
        self.actualGold = self.initialGold
        self.timelineGold = [self.initialGold]
        self.lastPrice = 0
        self.tokens = 0.0
        self.model = model
        
    def invest(self, prices):
        self.firstPrice = prices.iloc[0]
        self.lastPrice = prices.iloc[-1]
        pricesSeen = []
        for price in prices:
            
            pricesSeen.append(price)
            
            pred = self.model.make_next_prediction(pricesSeen)
            
            if pred == 0: # Price decrease prediction -> Sell
                self.sell(price)
                
            else: # Price increase prediction -> Buy
                self.buy(price)
                
        self.convert_balance_to_tokens()
            
    def sell(self, price):
        self.actualGold += price
        self.timelineGold.append(self.actualGold)
        
    def buy(self, price):
        self.actualGold -= price
        self.timelineGold.append(self.actualGold)
        
    def convert_balance_to_tokens(self):
            self.tokens = abs(self.actualGold) / self.lastPrice
        
    def get_investment_variation_mean(self):
        variations = []
        for previous, current in zip(self.timelineGold, self.timelineGold[1:]):
            try:
                variation = abs((current - previous) / previous) * 100.0
            except ZeroDivisionError:
                variation = 0
                
            variations.append(variation)
            
        return np.mean(variations)
    
    def get_dollars_for_final_balance(self):
        return self.tokens * 15
    
    def get_initial_dollars_investment(self):
        wowTokens = self.initialGold / self.firstPrice
    
        return wowTokens * 20.0

In [29]:
initialGold = 50000

### Buy-and-Hold Strategy Investment

In [30]:
def get_investment_variation_mean(initialGold, timelineGold):
        variations = []
        for previous, current in zip(timelineGold, timelineGold[1:]):
            try:
                variation = abs((current - previous) / previous) * 100.0
            except ZeroDivisionError:
                variation = 0
                
            variations.append(variation)
            
        return np.mean(variations)
    
def get_dollars_for_final_balance(finalBalance, lastPrice):
    wowTokens = finalBalance / lastPrice
        
    return wowTokens * 15.0

def get_initial_dollars_investment(initialGold, firstPrice):
    wowTokens = initialGold / firstPrice
    
    return wowTokens * 20.0

In [31]:
finalGoldNA = initialGold + dataNA['price'].iloc[-1]
buyAndHoldNABalance = finalGoldNA - dataNA['price'].iloc[0]
buyAndHoldNAVariation = get_investment_variation_mean(initialGold, dataNA['price'])
buyAndHoldNAInvestment = get_initial_dollars_investment(initialGold, dataNA['price'].iloc[0])
buyAndHoldNADollars = get_dollars_for_final_balance(finalGoldNA, dataNA['price'].iloc[-1])

In [32]:
finalGoldCN = initialGold + dataCN['price'].iloc[-1]
buyAndHoldCNBalance = finalGoldCN - dataCN['price'].iloc[0]
buyAndHoldCNVariation = get_investment_variation_mean(initialGold, dataCN['price'])
buyAndHoldCNInvestment = get_initial_dollars_investment(initialGold, dataCN['price'].iloc[0])
buyAndHoldCNDollars = get_dollars_for_final_balance(finalGoldCN, dataCN['price'].iloc[-1])

In [33]:
finalGoldEU = initialGold + dataEU['price'].iloc[-1]
buyAndHoldEUBalance = finalGoldEU - dataEU['price'].iloc[0]
buyAndHoldEUVariation = get_investment_variation_mean(initialGold, dataEU['price'])
buyAndHoldEUInvestment = get_initial_dollars_investment(initialGold, dataEU['price'].iloc[0])
buyAndHoldEUDollars = get_dollars_for_final_balance(finalGoldEU, dataEU['price'].iloc[-1])

In [34]:
finalGoldKR = initialGold + dataKR['price'].iloc[-1]
buyAndHoldKRBalance = finalGoldKR - dataKR['price'].iloc[0]
buyAndHoldKRVariation = get_investment_variation_mean(initialGold, dataKR['price'])
buyAndHoldKRInvestment = get_initial_dollars_investment(initialGold, dataKR['price'].iloc[0])
buyAndHoldKRDollars = get_dollars_for_final_balance(finalGoldKR, dataKR['price'].iloc[-1])

In [35]:
finalGoldTW = initialGold + dataTW['price'].iloc[-1]
buyAndHoldTWBalance = finalGoldTW - dataTW['price'].iloc[0]
buyAndHoldTWVariation = get_investment_variation_mean(initialGold, dataTW['price'])
buyAndHoldTWInvestment = get_initial_dollars_investment(initialGold, dataTW['price'].iloc[0])
buyAndHoldTWDollars = get_dollars_for_final_balance(finalGoldTW, dataTW['price'].iloc[-1])

In [36]:
print('-'*40)
print('Buy-and-Hold Strategy')
print('-'*40)
print('North America\n')
print('Final Gold Balance: %d' %buyAndHoldNABalance)
print('Gold Mean Variation: %.2f%%' %buyAndHoldNAVariation)
print('Dollars Initial Investment: $%.2f USD' %buyAndHoldNAInvestment)
print('Dollars Profit Investment: $%.2f USD' %buyAndHoldNADollars)
print('Dollars Total Balance: $%.2f USD' %(buyAndHoldNADollars-buyAndHoldNAInvestment))
print('-'*40)
print('China\n')
print('Final Gold Balance: %d' %buyAndHoldCNBalance)
print('Gold Mean Variation: %.2f%%' %buyAndHoldCNVariation)
print('Dollars Initial Investment: $%.2f USD' %buyAndHoldCNInvestment)
print('Dollars Profit Investment: $%.2f USD' %buyAndHoldCNDollars)
print('Dollars Total Balance: $%.2f USD' %(buyAndHoldCNDollars-buyAndHoldCNInvestment))
print('-'*40)
print('Europe\n')
print('Final Gold Balance: %d' %buyAndHoldEUBalance)
print('Gold Mean Variation: %.2f%%' %buyAndHoldEUVariation)
print('Dollars Initial Investment: $%.2f USD' %buyAndHoldEUInvestment)
print('Dollars Profit Investment: $%.2f USD' %buyAndHoldEUDollars)
print('Dollars Total Balance: $%.2f USD' %(buyAndHoldEUDollars-buyAndHoldEUInvestment))
print('-'*40)
print('South Korea\n')
print('Final Gold Balance: %d' %buyAndHoldKRBalance)
print('Gold Mean Variation: %.2f%%' %buyAndHoldKRVariation)
print('Dollars Initial Investment: $%.2f USD' %buyAndHoldKRInvestment)
print('Dollars Profit Investment: $%.2f USD' %buyAndHoldKRDollars)
print('Dollars Total Balance: $%.2f USD' %(buyAndHoldKRDollars-buyAndHoldKRInvestment))
print('-'*40)
print('Taiwan\n')
print('Final Gold Balance: %d' %buyAndHoldTWBalance)
print('Gold Mean Variation: %.2f%%' %buyAndHoldTWVariation)
print('Dollars Initial Investment: $%.2f USD' %buyAndHoldTWInvestment)
print('Dollars Profit Investment: $%.2f USD' %buyAndHoldTWDollars)
print('Dollars Total Balance: $%.2f USD' %(buyAndHoldTWDollars-buyAndHoldTWInvestment))
print('-'*40)

----------------------------------------
Buy-and-Hold Strategy
----------------------------------------
North America

Final Gold Balance: 126087
Gold Mean Variation: 1.68%
Dollars Initial Investment: $33.33 USD
Dollars Profit Investment: $22.07 USD
Dollars Total Balance: $-11.26 USD
----------------------------------------
China

Final Gold Balance: 214603
Gold Mean Variation: 1.61%
Dollars Initial Investment: $20.57 USD
Dollars Profit Investment: $18.52 USD
Dollars Total Balance: $-2.06 USD
----------------------------------------
Europe

Final Gold Balance: 190346
Gold Mean Variation: 1.89%
Dollars Initial Investment: $28.57 USD
Dollars Profit Investment: $19.28 USD
Dollars Total Balance: $-9.29 USD
----------------------------------------
South Korea

Final Gold Balance: 213514
Gold Mean Variation: 2.28%
Dollars Initial Investment: $8.10 USD
Dollars Profit Investment: $17.61 USD
Dollars Total Balance: $9.52 USD
----------------------------------------
Taiwan

Final Gold Balance: 62

### Basic Prediction Model Investment

In [37]:
basicInvestmentNA = Investment(BasicModel(), initialGold)
basicInvestmentNA.invest(dataNA['price'])

basicInvestmentCN = Investment(BasicModel(), initialGold)
basicInvestmentCN.invest(dataCN['price'])

basicInvestmentEU = Investment(BasicModel(), initialGold)
basicInvestmentEU.invest(dataEU['price'])

basicInvestmentKR = Investment(BasicModel(), initialGold)
basicInvestmentKR.invest(dataKR['price'])

basicInvestmentTW = Investment(BasicModel(), initialGold)
basicInvestmentTW.invest(dataTW['price'])

In [38]:
print('-'*40)
print('Basic Prediction Model Strategy')
print('-'*40)
print('North America\n')
print('Final Gold Balance: %d' %basicInvestmentNA.actualGold)
print('Gold Mean Variation: %.2f%%' %basicInvestmentNA.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %basicInvestmentNA.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %basicInvestmentNA.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(basicInvestmentNA.get_dollars_for_final_balance()-basicInvestmentNA.get_initial_dollars_investment()))
print('-'*40)
print('China\n')
print('Final Gold Balance: %d' %basicInvestmentCN.actualGold)
print('Gold Mean Variation: %.2f%%' %basicInvestmentCN.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %basicInvestmentCN.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %basicInvestmentCN.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(basicInvestmentCN.get_dollars_for_final_balance()-basicInvestmentCN.get_initial_dollars_investment()))
print('-'*40)
print('Europe\n')
print('Final Gold Balance: %d' %basicInvestmentEU.actualGold)
print('Gold Mean Variation: %.2f%%' %basicInvestmentEU.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %basicInvestmentEU.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %basicInvestmentEU.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(basicInvestmentEU.get_dollars_for_final_balance()-basicInvestmentEU.get_initial_dollars_investment()))
print('-'*40)
print('South Korea\n')
print('Final Gold Balance: %d' %basicInvestmentKR.actualGold)
print('Gold Mean Variation: %.2f%%' %basicInvestmentKR.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %basicInvestmentKR.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %basicInvestmentKR.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(basicInvestmentKR.get_dollars_for_final_balance()-basicInvestmentKR.get_initial_dollars_investment()))
print('-'*40)
print('Taiwan\n')
print('Final Gold Balance: %d' %basicInvestmentTW.actualGold)
print('Gold Mean Variation: %.2f%%' %basicInvestmentTW.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %basicInvestmentTW.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %basicInvestmentTW.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(basicInvestmentTW.get_dollars_for_final_balance()-basicInvestmentTW.get_initial_dollars_investment()))
print('-'*40)

----------------------------------------
Basic Prediction Model Strategy
----------------------------------------
North America

Final Gold Balance: -24281214
Gold Mean Variation: 7.73%
Dollars Initial Investment: $33.33 USD
Dollars Profit Investment: $3433.20 USD
Dollars Total Balance: $3399.87 USD
----------------------------------------
China

Final Gold Balance: 24256984
Gold Mean Variation: 9.44%
Dollars Initial Investment: $20.57 USD
Dollars Profit Investment: $1706.58 USD
Dollars Total Balance: $1686.01 USD
----------------------------------------
Europe

Final Gold Balance: 1575091
Gold Mean Variation: 49.78%
Dollars Initial Investment: $28.57 USD
Dollars Profit Investment: $134.74 USD
Dollars Total Balance: $106.17 USD
----------------------------------------
South Korea

Final Gold Balance: 1008178
Gold Mean Variation: 61.96%
Dollars Initial Investment: $8.10 USD
Dollars Profit Investment: $52.69 USD
Dollars Total Balance: $44.59 USD
----------------------------------------
T

### Random Prediction Model Investment


In [39]:
randomInvestmentNA = Investment(RandomModel(), initialGold)
randomInvestmentNA.invest(dataNA['price'])

randomInvestmentCN = Investment(RandomModel(), initialGold)
randomInvestmentCN.invest(dataCN['price'])

randomInvestmentEU = Investment(RandomModel(), initialGold)
randomInvestmentEU.invest(dataEU['price'])

randomInvestmentKR = Investment(RandomModel(), initialGold)
randomInvestmentKR.invest(dataKR['price'])

randomInvestmentTW = Investment(RandomModel(), initialGold)
randomInvestmentTW.invest(dataTW['price'])

In [40]:
print('-'*40)
print('Random Prediction Model Strategy')
print('-'*40)
print('North America\n')
print('Final Gold Balance: %d' %randomInvestmentNA.actualGold)
print('Gold Mean Variation: %.2f%%' %randomInvestmentNA.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %randomInvestmentNA.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %randomInvestmentNA.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(randomInvestmentNA.get_dollars_for_final_balance()-randomInvestmentNA.get_initial_dollars_investment()))
print('-'*40)
print('China\n')
print('Final Gold Balance: %d' %randomInvestmentCN.actualGold)
print('Gold Mean Variation: %.2f%%' %randomInvestmentCN.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %randomInvestmentCN.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %randomInvestmentCN.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(randomInvestmentCN.get_dollars_for_final_balance()-randomInvestmentCN.get_initial_dollars_investment()))
print('-'*40)
print('Europe\n')
print('Final Gold Balance: %d' %randomInvestmentEU.actualGold)
print('Gold Mean Variation: %.2f%%' %randomInvestmentEU.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %randomInvestmentEU.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %randomInvestmentEU.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(randomInvestmentEU.get_dollars_for_final_balance()-randomInvestmentEU.get_initial_dollars_investment()))
print('-'*40)
print('South Korea\n')
print('Final Gold Balance: %d' %randomInvestmentKR.actualGold)
print('Gold Mean Variation: %.2f%%' %randomInvestmentKR.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %randomInvestmentKR.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %randomInvestmentKR.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(randomInvestmentKR.get_dollars_for_final_balance()-randomInvestmentKR.get_initial_dollars_investment()))
print('-'*40)
print('Taiwan\n')
print('Final Gold Balance: %d' %randomInvestmentTW.actualGold)
print('Gold Mean Variation: %.2f%%' %randomInvestmentTW.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %randomInvestmentTW.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %randomInvestmentTW.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(randomInvestmentTW.get_dollars_for_final_balance()-randomInvestmentTW.get_initial_dollars_investment()))
print('-'*40)

----------------------------------------
Random Prediction Model Strategy
----------------------------------------
North America

Final Gold Balance: 13567146
Gold Mean Variation: 9.44%
Dollars Initial Investment: $33.33 USD
Dollars Profit Investment: $1918.30 USD
Dollars Total Balance: $1884.97 USD
----------------------------------------
China

Final Gold Balance: 8490706
Gold Mean Variation: 22.04%
Dollars Initial Investment: $20.57 USD
Dollars Profit Investment: $597.36 USD
Dollars Total Balance: $576.78 USD
----------------------------------------
Europe

Final Gold Balance: -24169519
Gold Mean Variation: 42.94%
Dollars Initial Investment: $28.57 USD
Dollars Profit Investment: $2067.59 USD
Dollars Total Balance: $2039.01 USD
----------------------------------------
South Korea

Final Gold Balance: 33025182
Gold Mean Variation: 3.17%
Dollars Initial Investment: $8.10 USD
Dollars Profit Investment: $1725.83 USD
Dollars Total Balance: $1717.74 USD
------------------------------------

### Simple Moving Average Prediction Model Investment

In [41]:
smaInvestmentNA = Investment(SMAModel(10, 50), initialGold)
smaInvestmentNA.invest(dataNA['price'])

smaInvestmentCN = Investment(SMAModel(10, 50), initialGold)
smaInvestmentCN.invest(dataCN['price'])

smaInvestmentEU = Investment(SMAModel(10, 50), initialGold)
smaInvestmentEU.invest(dataEU['price'])

smaInvestmentKR = Investment(SMAModel(10, 50), initialGold)
smaInvestmentKR.invest(dataKR['price'])

smaInvestmentTW = Investment(SMAModel(10, 50), initialGold)
smaInvestmentTW.invest(dataTW['price'])

In [42]:
print('-'*40)
print('SMA Prediction Model Strategy')
print('-'*40)
print('North America\n')
print('Final Gold Balance: %d' %smaInvestmentNA.actualGold)
print('Gold Mean Variation: %.2f%%' %smaInvestmentNA.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %smaInvestmentNA.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %smaInvestmentNA.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(smaInvestmentNA.get_dollars_for_final_balance()-smaInvestmentNA.get_initial_dollars_investment()))
print('-'*40)
print('China\n')
print('Final Gold Balance: %d' %smaInvestmentCN.actualGold)
print('Gold Mean Variation: %.2f%%' %smaInvestmentCN.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %smaInvestmentCN.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %smaInvestmentCN.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(smaInvestmentCN.get_dollars_for_final_balance()-smaInvestmentCN.get_initial_dollars_investment()))
print('-'*40)
print('Europe\n')
print('Final Gold Balance: %d' %smaInvestmentEU.actualGold)
print('Gold Mean Variation: %.2f%%' %smaInvestmentEU.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %smaInvestmentEU.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %smaInvestmentEU.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(smaInvestmentEU.get_dollars_for_final_balance()-smaInvestmentEU.get_initial_dollars_investment()))
print('-'*40)
print('South Korea\n')
print('Final Gold Balance: %d' %smaInvestmentKR.actualGold)
print('Gold Mean Variation: %.2f%%' %smaInvestmentKR.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %smaInvestmentKR.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %smaInvestmentKR.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(smaInvestmentKR.get_dollars_for_final_balance()-smaInvestmentKR.get_initial_dollars_investment()))
print('-'*40)
print('Taiwan\n')
print('Final Gold Balance: %d' %smaInvestmentTW.actualGold)
print('Gold Mean Variation: %.2f%%' %smaInvestmentTW.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %smaInvestmentTW.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %smaInvestmentTW.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(smaInvestmentTW.get_dollars_for_final_balance()-smaInvestmentTW.get_initial_dollars_investment()))
print('-'*40)

----------------------------------------
SMA Prediction Model Strategy
----------------------------------------
North America

Final Gold Balance: -100762550
Gold Mean Variation: 0.66%
Dollars Initial Investment: $33.33 USD
Dollars Profit Investment: $14247.16 USD
Dollars Total Balance: $14213.82 USD
----------------------------------------
China

Final Gold Balance: -232962666
Gold Mean Variation: 2.17%
Dollars Initial Investment: $20.57 USD
Dollars Profit Investment: $16389.89 USD
Dollars Total Balance: $16369.32 USD
----------------------------------------
Europe

Final Gold Balance: -179589357
Gold Mean Variation: 0.54%
Dollars Initial Investment: $28.57 USD
Dollars Profit Investment: $15363.00 USD
Dollars Total Balance: $15334.43 USD
----------------------------------------
South Korea

Final Gold Balance: -110944510
Gold Mean Variation: 5.97%
Dollars Initial Investment: $8.10 USD
Dollars Profit Investment: $5797.75 USD
Dollars Total Balance: $5789.65 USD
-------------------------

### Exponencial Moving Average Prediction Model Investment

In [43]:
emaInvestmentNA = Investment(EMAModel(10, 50), initialGold)
emaInvestmentNA.invest(dataNA['price'])

emaInvestmentCN = Investment(EMAModel(10, 50), initialGold)
emaInvestmentCN.invest(dataCN['price'])

emaInvestmentEU = Investment(EMAModel(10, 50), initialGold)
emaInvestmentEU.invest(dataEU['price'])

emaInvestmentKR = Investment(EMAModel(10, 50), initialGold)
emaInvestmentKR.invest(dataKR['price'])

emaInvestmentTW = Investment(EMAModel(10, 50), initialGold)
emaInvestmentTW.invest(dataTW['price'])

In [44]:
print('-'*40)
print('EMA Prediction Model Strategy')
print('-'*40)
print('North America\n')
print('Final Gold Balance: %d' %emaInvestmentNA.actualGold)
print('Gold Mean Variation: %.2f%%' %emaInvestmentNA.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %emaInvestmentNA.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %emaInvestmentNA.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(emaInvestmentNA.get_dollars_for_final_balance()-emaInvestmentNA.get_initial_dollars_investment()))
print('-'*40)
print('China\n')
print('Final Gold Balance: %d' %emaInvestmentCN.actualGold)
print('Gold Mean Variation: %.2f%%' %emaInvestmentCN.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %emaInvestmentCN.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %emaInvestmentCN.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(emaInvestmentCN.get_dollars_for_final_balance()-emaInvestmentCN.get_initial_dollars_investment()))
print('-'*40)
print('Europe\n')
print('Final Gold Balance: %d' %emaInvestmentEU.actualGold)
print('Gold Mean Variation: %.2f%%' %emaInvestmentEU.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %emaInvestmentEU.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %emaInvestmentEU.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(emaInvestmentEU.get_dollars_for_final_balance()-emaInvestmentEU.get_initial_dollars_investment()))
print('-'*40)
print('South Korea\n')
print('Final Gold Balance: %d' %emaInvestmentKR.actualGold)
print('Gold Mean Variation: %.2f%%' %emaInvestmentKR.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %emaInvestmentKR.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %emaInvestmentKR.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(emaInvestmentKR.get_dollars_for_final_balance()-emaInvestmentKR.get_initial_dollars_investment()))
print('-'*40)
print('Taiwan\n')
print('Final Gold Balance: %d' %emaInvestmentTW.actualGold)
print('Gold Mean Variation: %.2f%%' %emaInvestmentTW.get_investment_variation_mean())
print('Dollars Initial Investment: $%.2f USD' %emaInvestmentTW.get_initial_dollars_investment())
print('Dollars Profit Investment: $%.2f USD' %emaInvestmentTW.get_dollars_for_final_balance())
print('Dollars Total Balance: $%.2f USD' %(emaInvestmentTW.get_dollars_for_final_balance()-emaInvestmentTW.get_initial_dollars_investment()))
print('-'*40)

----------------------------------------
EMA Prediction Model Strategy
----------------------------------------
North America

Final Gold Balance: -25657280
Gold Mean Variation: 1.07%
Dollars Initial Investment: $33.33 USD
Dollars Profit Investment: $3627.77 USD
Dollars Total Balance: $3594.44 USD
----------------------------------------
China

Final Gold Balance: -25767816
Gold Mean Variation: 4.92%
Dollars Initial Investment: $20.57 USD
Dollars Profit Investment: $1812.87 USD
Dollars Total Balance: $1792.30 USD
----------------------------------------
Europe

Final Gold Balance: -58370907
Gold Mean Variation: 2.37%
Dollars Initial Investment: $28.57 USD
Dollars Profit Investment: $4993.35 USD
Dollars Total Balance: $4964.78 USD
----------------------------------------
South Korea

Final Gold Balance: -25559282
Gold Mean Variation: 14.73%
Dollars Initial Investment: $8.10 USD
Dollars Profit Investment: $1335.68 USD
Dollars Total Balance: $1327.58 USD
----------------------------------

## Conclusão

O modelo de SMA (Simple Moving Average) foi o que obteve maiores resultados com um período de investimento de 3 anos (2015-2018), em média. Porém, esse lucro está condicionado ao jogador conseguir juntar todo a quantidade de gold do balanço final dele para conseguir comprar a moeda.

Portanto, mesmo sendo o melhor modelo de se prever a tendência da moeda, o jogador teria que gastar muito tempo no jogo para conseguir juntar uma quantidade de gold necessária.

Para se ter uma ideia, jogadores que realizam o "farm" de gold no jogo (jogam com a principal função de juntar gold), fazem, em média, 200k por mês de jogo. Portanto, para alcançar, por exemplo, o lucro na região da China analizado anteriormente, o jogador teria que ficar por conta desse "farm" por, aproximadamente, $1164.81$ meses ($97$ anos), tornando essa tarefa quase impossível de ser completada em um tempo de vida hábil.

Desse modo, essa simulação demonstra apenas o total potencial de se ganhar apenas investindo na moeda WoWToken do jogo World of Warcraft em um período considerado relativamente curto para investimentos (3 anos), mas é impossível de se ter uma quantidade de recursos no jogo (gold) necessária para realizar a compra de todas essas moedas.